In [1]:
!apt-get install -y git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [2]:
! rm -rf NLP-project
! git clone https://github.com/ReviBa/NLP-project
! cp -rf NLP-project/* .

Cloning into 'NLP-project'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 98 (delta 48), reused 65 (delta 20), pack-reused 0
Receiving objects: 100% (98/98), 3.65 MiB | 19.99 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [3]:
! pip install --q -r /content/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:

In [4]:
from datasets import load_dataset
import pandas as pd
import warnings
import torch

Set warnings filter to ignore:

In [5]:
warnings.filterwarnings('ignore')


In [6]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
assert DEVICE == 'cuda'

Pre process dialy-dialog

In [7]:
dataset = load_dataset("daily_dialog")
!git clone https://github.com/iitmnlp/Dialogue-Evaluation-with-BERT.git

Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Cloning into 'Dialogue-Evaluation-with-BERT'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 82 (delta 13), reused 40 (delta 6), pack-reused 0
Receiving objects: 100% (82/82), 6.51 MiB | 3.94 MiB/s, done.
Resolving deltas: 100% (13/13), done.


to create a lot of data, we split each dialog into many mini- dialogs as possible. (there are a lot of questions and responses in each dialog) In addition, we replace the speaker names with "Person1" and "Person2". because the name are not relevent for the training

In [8]:
qa_df = pd.DataFrame(columns=['question', 'answer'])

for row in dataset['train']['dialog']:
  i = 2
  while i <= len(row):
    dialog = row[:i]
    context = dialog[:-1]
    answer = dialog[-1]
    context_formatted = ""
    for idx, sentence in enumerate(context):
      speaker_n = (idx % 2) + 1
      context_formatted += "Person" + str(speaker_n) + ": " + sentence
    i+=2
    answer_formatted = "Person2: " + answer
    new_row = {'question': context_formatted, 'answer': answer_formatted}
    qa_df = qa_df.append(new_row, ignore_index=True)

In [9]:
qa_df

,question,answer
0,"Person1: Say , Jim , how about going for a few...",Person2: You know that is tempting but is rea...
1,"Person1: Say , Jim , how about going for a few...",Person2: Do you really think so ? I don't . I...
2,"Person1: Say , Jim , how about going for a few...",Person2: I suggest a walk over to the gym whe...
3,"Person1: Say , Jim , how about going for a few...",Person2: Sounds great to me ! If they are wil...
4,"Person1: Say , Jim , how about going for a few...",Person2: All right .
...,...,...
41632,"Person1: Uncle Ben , how did the Forbidden Cit...",Person2: Exactly . Here is the Nine-dragon Sc...
41633,"Person1: Uncle Ben , how did the Forbidden Cit...","Person2: Sure , this will be a really good pi..."
41634,"Person1: May I help you , sir ?",Person2: I want a pair of locus .
41635,"Person1: May I help you , sir ? Person2: I wa...",Person2: I need size 41 .


In [10]:
qa_df.to_csv("daily_dialog_for_training.csv")